## Multi-Level Model (MLM)

### 1. Introduction
So far we have counted on having the ideal conditions for performing ANOVA: our groups were equal in number and, more importantly, our dataset was complete. We didn't have any missing values. As you may imagine, such perfect conditions are rather rare in evaluation research, particularly in longitudinal designs because several participants drop out. 

This scenario brings up challenges that ANOVA-related analysis approaches can't deal with in a straightforward way. ANOVA would solve the missing values problem by removing of cases with missing values. Remember, however, that measurements re costly and a small sample reduces statistical power. You or the company/hospital/practice spent so much time testing participants and you do not want to dispose of valuable data that has been acquired. Instead, you can use an alternative to ANOVA that is able to reliably model the information that is available. 

MLMs are not only great because they can deal with missing values, but also because:

* MLMs can estimate individual differences in change via the variance of random slopes. Remember: Earlier, researchers calculated the difference score to investigate individual differences in change, but the approach is biased by **variance restriction of the difference score**.

* MLMs also estimate individual differences in starting values (pre-intervention scores).

* The fixed effects will provide an estimate of the average starting value of the outcome and its change. 

* Interaction effects between levels (i.e., cross-level interactions) will inform us whether the change and individual differences in change depend on the group (treatment or control) and on further person specific variables like emotional intelligence in our example (variable "EM" in data frame). 

In the following exercise, we apply the `lmer()` function in R. We fit a model by regressing depression scores onto *time*. As the *random* part of the function, we include "id" to denote the observational units within which the measurements are clustered (contextual variable). 

In [2]:
#######################################################
## Specify R environment for rpy2
#import os
#os.environ['R_HOME'] = r'C:/Program Files/R/R-4.4.3'  # Replace with your R path
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects import Formula
import contextlib
# Ipython extension for plotting
%load_ext rpy2.ipython
########################################################

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display

### 2. Computation
**Summary:**
`library(lme4)`
`lmer(Dependent variable ~ fixed-effects terms + random-effects terms, data = data.frame)`

A) **Fixed-Effects Terms**: all overall effects and interactions, i.e. 

     `predictor 1 + predictor 2 + predictor 1 * predictor 2…`

    Note that these predictors can be **Level-1** or **Level-2** variables.

B) **Random-Effects Terms**: 

    i. Only intercepts but not the slopes are random: 
    
        `(1 | Level-2 indicator variable)`
    
    ii. Only slopes but not intercepts are random: 
    
        `(0 + Level-1 variable | Level-2 indicator variable)`
    
    iii. Both slopes and intercepts, are random: 
    
          `(1 + Level-1 variable | Level-2 indicator variable)`

In [11]:
# Read the CSV file using a relative path
dep_long = pd.read_csv("../MLM_and_CSM/Datasets/dep_long.csv")

# Convert specific columns to categorical data types
dep_long['id'] = dep_long['id'].astype('category')


# Reorder the levels of the 'time' and 'group' factor
dep_long['time'] = pd.Categorical(dep_long['time'], categories=['pre', 'post'], ordered=True)
dep_long['group'] = pd.Categorical(dep_long['group'], categories=[0, 1], ordered=True)

print(dep_long.head())



  id group  EM  change  time  score
0  1     0  25      42   pre     34
1  1     0  25      42  post     76
2  2     0  34     -36   pre     87
3  2     0  34     -36  post     51
4  3     0  15       3   pre     68


In [4]:
# Activate pandas2ri for automatic conversion
pandas2ri.activate()

# Check if the lavaan package is installed, and install it if not
ro.r('''
if (!requireNamespace("lme4", quietly = TRUE)) {
    install.packages("lme4", repos = "http://cran.us.r-project.org")
}
''')

# Import the lavaan package in R
ro.r('library(lme4)')

# Load the dataset into R
ro.globalenv['dep_long'] = pandas2ri.py2rpy(dep_long)

# Fit the model
ro.r('mu <- lmer(score ~ time + (1|id), data = dep_long)')

# Display the summary of the SEM model
summary = ro.r('summary(mu)')
print(summary)

R[write to console]: Loading required package: Matrix



Linear mixed model fit by REML ['lmerMod']
Formula: score ~ time + (1 | id)
   Data: dep_long

REML criterion at convergence: 1723.2

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.74851 -0.72422 -0.03171  0.71407  2.10716 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept)  14.53    3.812  
 Residual             321.14   17.920  
Number of obs: 200, groups:  id, 100

Fixed effects:
            Estimate Std. Error t value
(Intercept)   53.735      1.323  40.608
time.L       -10.034      1.792  -5.599

Correlation of Fixed Effects:
       (Intr)
time.L 0.000 



This initial multilevel model focuses on **Level 1 (within-person) changes** in depression scores over time. Key findings from this model (`score ~ time.L + (1 | id)`) are:
*(Assuming `time.L` is the variable name from your output for the time predictor)*

* **Fixed Intercept (Average Baseline):** The estimated average depression score at the initial time point (when `time.L` is at its reference) is 53.735.
* **Fixed Slope for `time.L` (Average Within-Person Change):** On average, depression scores *within individuals* decrease by 10.034 units for each unit increase in `time.L` (from pre- to post-intervention). This effect of `time.L` is statistically significant ($t = -5.599$).

Collectively, these fixed effects describe the average Level 1 (within-person) trend. This can be represented by the equation:
**`Predicted Score ≈ 53.735 - 10.034 * time.L`**
This equation shows the average predicted score based on time, before considering individual-specific deviations.

This model incorporates a **random intercept**, which means it accounts for the fact that individuals start with different baseline levels of depression. However, at this stage, the rate of change over time (the slope) is assumed to be the same for everyone.

To explore if these depression scores and their change over time differ based on group membership, we will now extend the model. We'll add a **Level 2 (between-person) predictor**, `group`, to examine its main effect. This new model will be named `mu2`.

In [ ]:
# Fit the model
ro.r('mu2 = lmer(score ~ time + group + (1 | id), data = dep_long)')

# Display the summary of the SEM model
summary = ro.r('summary(mu2)')
print(summary)

R[write to console]: boundary (singular) fit: see help('isSingular')



Linear mixed model fit by REML ['lmerMod']
Formula: score ~ time + group + (1 | id)
   Data: dep_long

REML criterion at convergence: 1689.7

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.15434 -0.82150  0.07783  0.80402  1.99809 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept)   0.0     0.00   
 Residual             289.8    17.02   
Number of obs: 200, groups:  id, 100

Fixed effects:
            Estimate Std. Error t value
(Intercept)   60.580      1.702  35.586
time.L       -10.034      1.702  -5.894
group1       -13.690      2.408  -5.686

Correlation of Fixed Effects:
       (Intr) time.L
time.L  0.000       
group1 -0.707  0.000
optimizer (nloptwrap) convergence code: 0 (OK)
boundary (singular) fit: see help('isSingular')



This multilevel model (`mu2`) now incorporates a **Level 2 (between-person) predictor**, `group1`, to help explain differences in depression scores, while still modeling **Level 1 (within-person) changes** over `time`. The model formula is `score ~ time + group + (1 | id)`.

**Key Observations and Findings:**

* **Fixed Effects:**
    * **`(Intercept)`: 60.580**
        * This is the estimated average depression score at the initial time point (`time` = 0) for the **reference group** (when `group` = 0).
    * **`time.L`: -10.034**
        * This is the estimated average **within-person change** (decrease) in depression score for each one-unit increase in `time.L` (from pre- to post-intervention), holding `group1` constant. This effect remains statistically significant ($t = -5.894$).
    * **`group1`: -13.690**
        * This is the estimated average difference in depression scores between the group represented by `group` (when `group` = 1) and the reference group (when `group` = 0), holding `time.L` constant.
        * Where `group1 = 1` is the intervention group and `group1 = 0` is the control group, this significant negative coefficient ($t = -5.686$) suggests that the intervention group has, on average, depression scores that are 13.690 points lower than the control group, across all time points (as there's no interaction with time in this model yet).

The average predicted score, now accounting for `group`, can be described by the equation:
**`Predicted Score ≈ 60.580 - 10.034 * time - 13.690 * group`**


This model adds the effect of the variable `group` as a fixed effect (-13.690). It can be interpreted as the estimated group difference on the **overall** depression. This is the difference in depression between groups, as demonstrated here:

In [8]:
# Group by 'group' and calculate the mean of 'score' for each group
means_gr = dep_long.groupby('group')['score'].mean().reset_index()
means_gr.columns = ['group', 'mean_group']

# Print the means for each group
print(means_gr)

# Calculate the difference between the means of the groups
group_diff = means_gr['mean_group'].iloc[1] - means_gr['mean_group'].iloc[0]

# Print the difference
print(group_diff)

  group  mean_group
0     0       60.58
1     1       46.89
-13.689999999999998


Thus far, we  modeled the main effects (i.e., the effect of time and group). However, because we are interested in knowing how the change in depression from pre- to post-therapy differs between the groups (interaction effect), the next step is to include the **cross-level interaction** term between these two variables (time and group). Remember, the cross-level interaction means that a between-person variable (group) is interacting with a within-person variable (time). More generally, the cross-level interaction refers to any scenario where variables of different levels interact. 

In [9]:
# Fit the model
ro.r('mu3 <- lmer(score ~ group + time + group * time + (1 |id), data = dep_long)')

# Display the summary of the SEM model
summary = ro.r('summary(mu3)')
print(summary)

Linear mixed model fit by REML ['lmerMod']
Formula: score ~ group + time + group * time + (1 | id)
   Data: dep_long

REML criterion at convergence: 1657.1

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.85334 -0.78091 -0.03413  0.79503  1.74924 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept)   6.033   2.456  
 Residual             246.093  15.687  
Number of obs: 200, groups:  id, 100

Fixed effects:
              Estimate Std. Error t value
(Intercept)     60.580      1.607  37.704
group1         -13.690      2.272  -6.025
time.L          -1.273      2.219  -0.574
group1:time.L  -17.522      3.137  -5.585

Correlation of Fixed Effects:
            (Intr) group1 time.L
group1      -0.707              
time.L       0.000  0.000       
group1:tm.L  0.000  0.000 -0.707



This multilevel model includes the **interaction effect** between `group` and `time` (`group * time`) to determine if the change in depression scores over time differs based on group membership. It still includes a random intercept for `id` to account for individual baseline differences. The model formula is `score ~ group * time + (1 | id)`.

**Key Observations and Findings:**

* **Fixed Effects (Average Effects):**
    * **`(Intercept)`: 60.580** ($t = 37.704$, $p < .001$)
        * This is the estimated average depression score at the initial time point (`time` = 0, or pre-) for the **reference group** (when `group` = 0, the Control group).
    * **`group1`: -13.690** ($t = -6.025$, $p < .001$)
        * This is the estimated difference in scores at the initial time point (`time.L` = 0) between the group `group`=1 (Intervention) and the reference group (`group`=0). The intervention group's score is significantly lower by about 13.69 points *at baseline*.
    * **`time.L`: -1.273** ($t = -0.574$, $p \approx 0.57$)
        * This is the estimated average change in score over time (from pre to post) **specifically for the reference group** (`group` = 0, Control group). This small decrease is **not statistically significant**, suggesting no significant change over time for the control group on average.
    * **`group1:time.L` (Interaction): -17.522** ($t = -5.585$, $p < .001$)
        * **This is the key finding for your research question.** This significant negative interaction indicates that the change in scores over time is significantly *different* for the `group`=1 group compared to the reference group.
        * Specifically, the slope (change over time) for the `group`=1 group (Intervention) is estimated to be 17.522 points *more negative* (a steeper decrease) than the slope for the reference group (Control).

The average predicted value for `Score` based on this model is given by the equation:
**`Predicted Score ≈ 60.580 - 13.690 * group - 1.273 * time - 17.522 * group * time`**
*(Where `group` is 0 or 1, and `time.L` represents the time variable coding, 0=pre, 1=post).*

We can even proceed to add EM as predictor:

In [12]:
# Fit the model
ro.r('mu4 <- lmer(score ~ group + time + group * time + EM + (1 |id), data = dep_long)')

# Display the summary of the SEM model
summary = ro.r('summary(mu4)')
print(summary)

Linear mixed model fit by REML ['lmerMod']
Formula: score ~ group + time + group * time + EM + (1 | id)
   Data: dep_long

REML criterion at convergence: 1657.5

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.93154 -0.79642 -0.08054  0.81016  1.66850 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept)   4.736   2.176  
 Residual             246.093  15.687  
Number of obs: 200, groups:  id, 100

Fixed effects:
              Estimate Std. Error t value
(Intercept)    56.2903     3.4320  16.402
group1        -13.6173     2.2614  -6.022
time.L         -1.2728     2.2185  -0.574
EM              0.1651     0.1169   1.412
group1:time.L -17.5221     3.1375  -5.585

Correlation of Fixed Effects:
            (Intr) group1 time.L EM    
group1      -0.349                     
time.L       0.000  0.000              
EM          -0.885  0.023  0.000       
group1:tm.L  0.000  0.000 -0.707  0.000



As in model "mu2", the **fixed** estimate for EM indicates its effect on the **overall** depression score. Because "EM" is a continuous variable, its estimate (0.16) is interpreted as a regression weight - the expected change in depression given a one-unit increase in EM.

Note: If you wanted to test whether the change in depression differs depending on EM, a further **cross-level interaction** between time and EM needs to be added to the model. We don't go over such a model, but the syntax to run it would be: 

In [15]:
# Fit the model
ro.r('mu5 <- lmer(score ~ group + time + group * time + EM + EM * time + (1 |id), data = dep_long)')

# Display the summary of the SEM model
summary = ro.r('summary(mu5)')
print(summary)

Linear mixed model fit by REML ['lmerMod']
Formula: score ~ group + time + group * time + EM + EM * time + (1 | id)
   Data: dep_long

REML criterion at convergence: 1658.8

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.84769 -0.76834 -0.09611  0.83698  1.70945 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept)   4.088   2.022  
 Residual             247.388  15.729  
Number of obs: 200, groups:  id, 100

Fixed effects:
              Estimate Std. Error t value
(Intercept)    56.2903     3.4320  16.402
group1        -13.6173     2.2614  -6.022
time.L         -4.2212     4.7753  -0.884
EM              0.1651     0.1169   1.412
group1:time.L -17.4722     3.1465  -5.553
time.L:EM       0.1135     0.1626   0.698

Correlation of Fixed Effects:
            (Intr) group1 time.L EM     gr1:.L
group1      -0.349                            
time.L       0.000  0.000                     
EM          -0.885  0.023  0.000              
group1:tm.L 

We hope that by now you are aware of the several advantages of MLM compared to more traditional approaches. Keep in mind that this is only a brief introduction to MLM in the context of longitudinal data analysis.